In [7]:
import numpy as np
import os
from ultralytics import YOLO
import cv2

In [34]:
class DetectedObject:
    def __init__(self, bbox: list, label: str, conf: float):
        self.bbox = bbox
        self.label = label
        self.conf = conf
        self.xmin = int(bbox[0])
        self.ymin = int(bbox[1])
        self.xmax = int(bbox[2])
        self.ymax = int(bbox[3])
        self.width = self.xmax - self.xmin
        self.height = self.ymax - self.ymin
        self.depth: float = 0.0

    def use_depth(self, depth: np.ndarray):
        self.depth = np.mean(depth[self.ymin:self.ymax, self.xmin:self.xmax])

    def __str__(self):
        return f"{self.label}-conf_{self.conf:.2f}-depth_{self.depth:.2f}-{self.width}x{self.height}"


In [43]:
import time


# get all objects in the image using given model
# and use depth to calculate the depth of each object
# sort the objects by xmin
def get_objects(image_path: str, depth_path: str, model: YOLO) -> list[DetectedObject]:
    results = model(image_path)[0].numpy()
    objects = []
    # if depth_path not exist, wait 5 seconds and try again
    while not os.path.exists(depth_path):
        print(f"Waiting for {depth_path} to be generated")
        time.sleep(5)
    depth_np = np.load(depth_path)
    for box in results.boxes:
        conf = box.conf[0]
        label = model.names[int(box.cls[0])]
        bbox = box.xyxy[0]
        obj = DetectedObject(bbox, label, conf)
        obj.use_depth(depth_np)
        objects.append(obj)
    objects.sort(key=lambda x: x.xmin)
    return objects

In [36]:
# Image to be processed
class Image:
    def __init__(self, image_path: str, depth_np_path: str, depth_img_path: str):
        self.image_path = image_path
        self.image_name = os.path.basename(image_path)
        self.depth_path = depth_np_path
        self.depth_img_path = depth_img_path
        self.depth_img_name = os.path.basename(depth_img_path)
        self.objects: list[DetectedObject] = []

    def detect_objects(self, model: YOLO):
        self.objects = get_objects(self.image_path, self.depth_path, model)

    # crop the image into objects
    # and save it to the given directory
    # using object's __str__ as the file name
    def crop_img(self, save_dir: str):
        # clear the directory
        for file in os.listdir(save_dir):
            os.remove(os.path.join(save_dir, file))
        img = cv2.imread(self.image_path)
        for i, obj in enumerate(self.objects):
            crop_img = img[obj.ymin:obj.ymax, obj.xmin:obj.xmax]
            cv2.imwrite(os.path.join(save_dir, f"{i}-{obj}.jpg"), crop_img)

    # draw the bounding box of each object on the image
    # on both the image and the depth image
    # and save it to the given directory
    # using image's name as the file name
    def draw_bbox(self, save_dir: str):
        img = cv2.imread(self.image_path)
        depth_img = cv2.imread(self.depth_img_path)
        color = (0, 255, 0)
        thickness = 2
        for obj in self.objects:
            cv2.rectangle(img, (obj.xmin, obj.ymin), (obj.xmax, obj.ymax), color, thickness)
            cv2.rectangle(depth_img, (obj.xmin, obj.ymin), (obj.xmax, obj.ymax), color, thickness)
        cv2.imwrite(os.path.join(save_dir, self.image_name), img)
        cv2.imwrite(os.path.join(save_dir, self.depth_img_name), depth_img)


In [37]:
# load yolo models
def load_models() -> dict[str, YOLO]:
    model8n = YOLO("yolov8n.pt")
    model8x = YOLO("yolov8x.pt")
    model8x6 = YOLO("yolov8x6.pt")
    models = {
        "yolov8n": model8n,
        "yolov8x": model8x,
        "yolov8x6": model8x6
    }
    return models

In [38]:
def main():
    # load model
    models = load_models()
    # load images
    image_dir = "../test"
    depth_dir = "../depth"
    results_base_dir = "../results"
    # for each image name, find the corresponding depth image and depth npy
    for image_file in os.listdir(image_dir):
        image_name = os.path.splitext(image_file)[0]
        image_path = os.path.join(image_dir, image_file)
        depth_dir_this_image = os.path.join(depth_dir, image_name)
        depth_img_path = os.path.join(depth_dir_this_image, image_name + ".jpg")
        depth_np_path = os.path.join(depth_dir_this_image, image_name + ".npy")
        print(f"Processing {image_path}: {depth_img_path}, {depth_np_path}")
        # create image object
        image = Image(image_path, depth_np_path, depth_img_path)
        # for each model, detect objects
        for model_name, model in models.items():
            print(f"Detecting objects using {model_name}")
            image.detect_objects(model)
            # crop the image into objects
            crop_dir = os.path.join(results_base_dir, model_name, image_name, "crop")
            os.makedirs(crop_dir, exist_ok=True)
            image.crop_img(crop_dir)
            # draw bounding box on the image
            draw_dir = os.path.join(results_base_dir, model_name, image_name)
            os.makedirs(draw_dir, exist_ok=True)
            image.draw_bbox(draw_dir)


In [45]:
if __name__ == "__main__":
    main()


image 1/1 /home/zkr/experiments/yolo/test/0000000001.png: 192x640 1 car, 1 bench, 41.4ms
Speed: 0.3ms preprocess, 41.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


Processing ../test/0000000001.png: ../depth/0000000001/0000000001.jpg, ../depth/0000000001/0000000001.npy
Detecting objects using yolov8n
Waiting for ../depth/0000000001/0000000001.npy to be generated
Waiting for ../depth/0000000001/0000000001.npy to be generated
Waiting for ../depth/0000000001/0000000001.npy to be generated
Detecting objects using yolov8x



image 1/1 /home/zkr/experiments/yolo/test/0000000001.png: 192x640 1 car, 1 bird, 1 chair, 840.7ms
Speed: 50.5ms preprocess, 840.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)


Detecting objects using yolov8x6



image 1/1 /home/zkr/experiments/yolo/test/0000000001.png: 384x1280 1 car, 2 birds, 2531.2ms
Speed: 1.5ms preprocess, 2531.2ms inference, 1.2ms postprocess per image at shape (1, 3, 1280, 1280)

image 1/1 /home/zkr/experiments/yolo/test/000587.png: 224x640 4 cars, 2 buss, 139.3ms
Speed: 6.8ms preprocess, 139.3ms inference, 4.2ms postprocess per image at shape (1, 3, 640, 640)


...

image 1/1 /home/zkr/experiments/yolo/test/000047.png: 448x1280 15 cars, 2895.9ms
Speed: 1.5ms preprocess, 2895.9ms inference, 1.2ms postprocess per image at shape (1, 3, 1280, 1280)

image 1/1 /home/zkr/experiments/yolo/test/000247.png: 224x640 5 cars, 2 trucks, 53.6ms
Speed: 0.5ms preprocess, 53.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)



Processing ../test/000247.png: ../depth/000247/000247.jpg, ../depth/000247/000247.npy
Detecting objects using yolov8n
Detecting objects using yolov8x


image 1/1 /home/zkr/experiments/yolo/test/000247.png: 224x640 1 bicycle, 4 cars, 2 buss, 2 trucks, 1403.5ms
Speed: 0.4ms preprocess, 1403.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)



Detecting objects using yolov8x6


image 1/1 /home/zkr/experiments/yolo/test/000247.png: 448x1280 1 bicycle, 3 cars, 4 trucks, 2908.3ms
Speed: 1.5ms preprocess, 2908.3ms inference, 2.0ms postprocess per image at shape (1, 3, 1280, 1280)
